In [1]:
from scapy.all import *
import numpy as np
import pandas as pd
import argparse
import os
from labels import mac_to_label

inputfile = "data.pcap"
outputfile = "data.csv"

In [2]:
#read the pcap file and extract the features for each packet
all_packets = rdpcap(inputfile)

In [3]:
results = []
for packet in all_packets:
    size = len(packet)
    try:
        proto = packet.proto
    except AttributeError:
        proto = 0
    try:
        sport = packet.sport
        dport = packet.dport
    except AttributeError:
        sport = 0
        dport = 0

    proto = int(proto)
    sport = int(sport)
    dport = int(dport)

    if "Ether" in packet:
        eth_dst = packet["Ether"].dst
        if eth_dst in mac_to_label:
            classification = mac_to_label[eth_dst]
        else:
            classification = "other"
    else:
        classification = "other"

    metric = [proto,sport,dport,classification]
    results.append(metric)
results = (np.array(results)).T

In [4]:
# store the features in the dataframe
dataframe = pd.DataFrame({'protocl':results[0],'src':results[1],'dst':results[2],'classfication':results[3]})
columns = ['protocl','src','dst','classfication']

# save the dataframe to the csv file, if not exsit, create one.
if os.path.exists(outputfile):
    dataframe.to_csv(outputfile,index=False,sep=',',mode='a',columns = columns, header=False)
else:
    dataframe.to_csv(outputfile,index=False,sep=',',columns = columns)